## 欠損値の補完　
- SimpleImputer

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# サンプルのデータセットを作成
df = pd.DataFrame(
    {
        "A": [1, np.nan, 3, 4, 5],
        "B": [6, 7, 8, np.nan, 10],
        "C": [11, 12, 13, 14, 15]
    }
)


# 平均値で欠損値を補完するインスタンスを作成する
imp = SimpleImputer(strategy="mean")
# 欠損値を補完
imp.fit(df)
imp.transform(df)

## カテゴリ変数のエンコーディング　
- LabelEncoder
- OneHotEncoder
- ColumnTransformer

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

df = pd.DataFrame(
    {
        "A": [1, 2, 3, 4, 5],
        "B": ["a", "b", "a", "b", "c"]
    }
)

# ラベルエンコーディング
# ラベルエンコーダのインスタンス化
le = LabelEncoder()
# 英語のa、b、cを1、2、3に変換
le.fit(df.loc[:, "B"])
le.transform(df.loc[:, "B"])

# One-hotエンコーディング
df_ohe = df.copy()
# ラベルエンコーダのインスタンス化
le = LabelEncoder()
# 英語のa、b、cを1、2、3に変換
df_ohe.loc[:, "B"] = le.fit_transform(df_ohe.loc[:, "B"])
# One-hotエンコーダのインスタンス化
ohe = ColumnTransformer([("OneHotEncoder",
                          OneHotEncoder(), [1])],
                          remainder = "passthrough")
# One-hotエンコーディング
df_ohe = ohe.fit_transform(df_ohe)
df_ohe

,A,B
0,1,a
1,2,b
2,3,a
3,4,b
4,5,c


array([0, 1, 0, 1, 2])

array([[1., 0., 0., 1.],
       [0., 1., 0., 2.],
       [1., 0., 0., 3.],
       [0., 1., 0., 4.],
       [0., 0., 1., 5.]])

## 特徴量の正規化

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# DataFrameを作成する
df = pd.DataFrame(
    {
        "A": [1, 2, 3, 4, 5],
        "B": [100, 200, 400, 500, 800]
    }
)
df

# 分散正規化のインスタンスを生成
stdsc = StandardScaler()
# 分散正規化を実行
stdsc.fit(df)
stdsc.transform(df)

# 最小最大正規化のインスタンスを生成
mmsc = MinMaxScaler()
# 最小最大正規化を実行
mmsc.fit(df)
mmsc.transform(df)

,A,B
0,1,100
1,2,200
2,3,400
3,4,500
4,5,800


# 分類
## サポートベクターマシン
## 決定木
## ランダムフォレスト